In [2]:
import pandas as pd

In [24]:
df = pd.read_csv('ads.csv', index_col=['date', 'ad_channel', 'country'])
df.head()

impressions  clicks  ad_spend
date       ad_channel country                               
2023-01-01 facebook   US               216     109    206.19
2023-01-02 instagram  Canada           203     200    145.62
           linkedin   France           123      90     68.09
                      Denmark          181     138     42.11
2023-01-03 linkedin   US               219     170     46.29

In [18]:
# работа с индексами из нескольких колонок

df.loc['2023-01-02'] #покажет строки, у которых индекс равен значению
df.loc['2023-01-02'].loc['linkedin'] #покажет подстроки с указанным 2-м индексом

,impressions,clicks,ad_spend,country
ad_channel,,,,
linkedin,123,90,68.09,France
linkedin,181,138,42.11,Denmark


In [22]:
# агрегирование серий

df['ad_spend'].sum() #сумма
df['ad_spend'].count() #количество строк
df['ad_spend'].min() #минимальное
df['ad_spend'].mean() #среднее ариф
df['ad_spend'].quantile(.25) #квантиль
df['ad_spend'].unique() #список уникальных значений
df['ad_spend'].nunique() #количество уникальных значений (общее число)
df['ad_spend'].idxmax() #максимальное значение серии и возвращает его индекс
                        # ниже два индекса, т.к. в таблц 2 индекса


('2023-02-10', 'twitter')

In [23]:
df.dtypes #смотрим типы данных колонок

impressions      int64
clicks           int64
ad_spend       float64
country         object
dtype: object

In [29]:
# агрегирование дата-фреймов

df.sum() #применит функцию к каждой колонке, к которой может это сделать
df.sum()['impressions'] #только к колонке impressions

df.sum(axis='rows') #по умолчанию агрегирование идёт вдоль строк
df.sum(axis='columns') #суммирует все значени в колонках НЕ индексах
                       #в примере ниже первые три колонки - индексы

date        ad_channel  country
2023-01-01  facebook    US         531.19
2023-01-02  instagram   Canada     548.62
            linkedin    France     281.09
                        Denmark    361.11
2023-01-03  linkedin    US         435.29
                                    ...  
2023-02-16  instagram   Brazil     380.05
2023-02-17  linkedin    US         346.26
            facebook    Brazil     753.65
2023-02-18  linkedin    Sweden     638.79
2023-02-19  instagram   Canada     460.26
Length: 98, dtype: float64

In [44]:
# группировка датафреймов

df.groupby('country')  # получаем объект похожий на датафрейм
                       # из которого мы можем извлекать данные
df.groupby('country').sum() #например так

df.groupby('country').sum() # группировка по колонке страна 
                            # с суммой по колонкам

df.groupby('country').mean()
df.groupby('ad_channel').sum()
df.groupby(['ad_channel', 'country']).sum() # передаём списко по каналу 
                                            # И стране со всеми колонками

df.groupby('country').agg(     #чаще используют функцию agg, чем обычные
    {'impressions': ['sum', 'min', 'max']} #применяем агрегацию с несколькоими параметрами      
)

df.groupby('country').agg(     #чаще используют функцию agg, чем обычные
    {'impressions': ['sum', 'count'], 
     'ad_spend': ['sum', 'mean']}
) # т.к. это словарь, то нужно делать новый ключ и к нему применять нужные функции

impressions       ad_spend            
                  sum count      sum        mean
country                                         
Argentina        1920     8  1079.93  134.991250
Brazil           2305     8  1382.99  172.873750
Canada           1772     7   799.25  114.178571
China             796     3   310.72  103.573333
Denmark          3030    10  1471.70  147.170000
France           1301     6   804.62  134.103333
Germany          2907    10  2048.31  204.831000
India            1461     5  1019.33  203.866000
Spain            2683     9  1192.63  132.514444
Sweden           3644    11  2187.33  198.848182
UK               2727    11  1718.75  156.250000
US               3090    10  2034.64  203.464000

In [47]:
df.groupby('country').agg(
    total_impressions = ('impressions', 'sum'), 
    av_daily_spend = ('ad_spend', 'mean') 
) #так можно только с одной функцией в кортеже

,total_impressions,av_daily_spend
country,,
Argentina,1920,134.991250
Brazil,2305,172.873750
Canada,1772,114.178571
China,796,103.573333
Denmark,3030,147.170000
France,1301,134.103333
Germany,2907,204.831000
India,1461,203.866000
Spain,2683,132.514444


In [58]:
rev_per_click = {
    'Argentina': 0.04,
    'France': 0.35,
    'UK': 0.4,
    'China': 0.07,
    'Denmark': 0.4,
    'US': 0.5,
    'Germany': 0.6,
    'Sweden': 0.56,
    'Canada': 0.3,
    'Spain': 0.34,
    'Brazil': 0.25,
    'India': 0.04,
}

def calculate_roi(group):
    country = group.reset_index()['country'].iloc[0] #взяли колонку и первое значение
    clicks = group['clicks'].sum() #агрегировали колонку клики внутри групп
    ad_spend = group['ad_spend'].sum() #агрегировали колонку эд спенд внутри групп
    revenue = clicks * rev_per_click[country] #взяли страны из списка выше
    roi = revenue / ad_spend
    return roi #передаём дата фрейм

df.groupby('country').apply(calculate_roi) # группируем по странам
                                           # и применяем функцию 

country
Argentina    0.051781
Brazil       0.280009
Canada       0.424148
China        0.150264
Denmark      0.624312
France       0.405409
Germany      0.590243
India        0.038849
Spain        0.557625
Sweden       0.743226
UK           0.493847
US           0.577252
dtype: float64

In [61]:
# сортировка
# .sort_index()
# .sort_values()

df.groupby('country').apply(calculate_roi).sort_values(ascending=False)
# сортировка серии по индексам, по убыванию


country
Sweden       0.743226
Denmark      0.624312
Germany      0.590243
US           0.577252
Spain        0.557625
UK           0.493847
Canada       0.424148
France       0.405409
Brazil       0.280009
China        0.150264
Argentina    0.051781
India        0.038849
dtype: float64

In [62]:
df.sort_values(by='ad_spend', ascending=False)
# сортировка дата фрейма по выбранной колонке с убыванием

,,,impressions,clicks,ad_spend
date,ad_channel,country,,,
2023-02-10,twitter,Denmark,467,372,416.98
2023-01-05,google,UK,405,298,400.22
2023-02-10,facebook,US,459,279,394.44
2023-01-19,google,US,449,434,393.04
2023-02-08,instagram,Germany,389,329,374.58
...,...,...,...,...,...
2023-01-21,google,Canada,151,133,35.37
2023-01-29,twitter,Germany,106,85,27.60
2023-01-13,linkedin,China,108,92,24.03


In [ ]:
(
    df
    .groupby('country') #метод агреггерации
    .apply(calculate_roi) #метод группировки
    .sort_values(ascending=False) #метод сортировки
) 
# код может быть бесконечно длинным и работать в скобках
# на каждом этапе мы получаем серию ИЛИ дата фрейм 
# к которому можем применить соотвествующий метод